In [1]:
import tensorflow as tf
import keras
import math
import numpy as np
import json

In [2]:
kernel_init = tf.keras.initializers.GlorotUniform()
bias_init = tf.keras.initializers.Constant(value=0.2)

def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = tf.keras.layers.Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = tf.keras.layers.Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = tf.keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [3]:
input_layer = tf.keras.layers.Input(shape=(48, 48, 1))

x = tf.keras.layers.Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = tf.keras.layers.Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = tf.keras.layers.Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = tf.keras.layers.MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Dense(units=64,activation="relu")(x)
x = tf.keras.layers.Dense(units=32,activation="relu")(x)
x = tf.keras.layers.Dense(units=16,activation="relu")(x)

x = tf.keras.layers.Dense(7, activation='softmax', name='output')(x)

model = tf.keras.models.Model(input_layer, x, name='inception_v1')

model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-09 18:48:56.841193: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-09 18:48:56.842635: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "inception_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 24, 24, 64)   3200        ['input_1[0][0]']                
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 12, 12, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                                
                                                                                                  
 conv_2a_3x3/1 (Conv2D)         (None, 12, 12, 64)   4160        ['max_pool_1_3x3/2[0][

In [4]:
epochs = 75
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.98
    epochs_drop = 10
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = tf.keras.optimizers.SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = tf.keras.callbacks.LearningRateScheduler(decay, verbose=1)

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=sgd, metrics=['accuracy'])

In [5]:
ds_path = r'/Users/heathsmith/repos/github/neuromorphic-computing/datasets/ck_plus_48.npz'

with np.load(ds_path) as data:
    images = data['images']
    labels = data['labels']
    classes = data['classes']
print(np.shape(images), np.shape(labels))

# constants
BATCH_SIZE = 16
_TRAIN = .80
_VAL = .10
_TEST = .10
SEED = 123

# create a dataset object and shuffle
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(len(images) * BATCH_SIZE, seed=SEED)
shuffle_images = []
shuffle_labels = []
for im, l in dataset.as_numpy_iterator():
    shuffle_images.append(im)
    shuffle_labels.append(l)
shuffle_images = np.asarray(shuffle_images)
shuffle_labels = np.asarray(shuffle_labels)
print(np.shape(shuffle_images))
print(len(dataset))

# create the slicing indices
TRAIN = round(len(dataset) * _TRAIN)
VAL = TRAIN + round(len(dataset) * _VAL)
TEST = VAL + round(len(dataset) * _TEST)
print(TRAIN, VAL, TEST)

x_train = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[0:TRAIN]))
x_train = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_train))
train_ds =  tf.data.Dataset.from_tensor_slices((x_train, shuffle_labels[0:TRAIN]))
train_ds = train_ds.cache()
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(x_train))

x_val = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[TRAIN:VAL]))
x_val = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_val))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, shuffle_labels[TRAIN:VAL]))
val_ds = val_ds.cache()
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size=BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE, drop_remainder=False)
print(np.shape(shuffle_images[TRAIN:VAL]))

x_test = list(map(lambda x: np.expand_dims(x, axis=2), shuffle_images[VAL:TEST]))
x_test = list(map(lambda x: np.array(x/255.0).astype(np.float32), x_test))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, shuffle_labels[VAL:TEST]))
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size=BATCH_SIZE,num_parallel_calls=tf.data.AUTOTUNE,drop_remainder=False)


(981, 48, 48) (981, 7)
(981, 48, 48)
981
785 883 981
(785, 48, 48, 1)
(98, 48, 48)


In [6]:


# add checkpoint callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    './inception.h5',
    monitor='val_accuracy',
    verbose=0,
    save_best=True,
    save_weights_only=False,
    mode='max'
)

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=BATCH_SIZE, callbacks=[lr_sc, checkpoint], verbose=1)



Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/75


2023-02-09 18:48:59.498240: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-09 18:48:59.502460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - ETA: 0s - loss: 1.8958 - accuracy: 0.2076

2023-02-09 18:49:03.723852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 5s 54ms/step - loss: 1.8958 - accuracy: 0.2076 - val_loss: 1.8406 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/75
50/50 [==============================] - 1s 28ms/step - loss: 1.8407 - accuracy: 0.2382 - val_loss: 1.8503 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/75
50/50 [==============================] - 1s 27ms/step - loss: 1.8292 - accuracy: 0.2459 - val_loss: 1.8424 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/75
50/50 [==============================] - 1s 27ms/step - loss: 1.8276 - accuracy: 0.2471 - val_loss: 1.8405 - val_accuracy: 0.3163 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/75
50/50 [==============================] - 1s 27ms/step - loss: 1.8185 - accuracy: 0.2662 - val_loss: 1.8422 - val_accuracy: 0.3163 - 

In [7]:
#json.dump(history.history, open('./inception_history.json', 'w'))
print(history.history)

{'loss': [1.8957782983779907, 1.8406598567962646, 1.8292031288146973, 1.8276331424713135, 1.8184856176376343, 1.8184311389923096, 1.8300845623016357, 1.8230899572372437, 1.8147598505020142, 1.8110053539276123, 1.8117446899414062, 1.810325026512146, 1.806899070739746, 1.8024274110794067, 1.8049015998840332, 1.8024941682815552, 1.7988388538360596, 1.7956660985946655, 1.7919889688491821, 1.7899972200393677, 1.7889490127563477, 1.7917004823684692, 1.7867155075073242, 1.7883198261260986, 1.7839419841766357, 1.7736341953277588, 1.7637509107589722, 1.7498068809509277, 1.7315378189086914, 1.7112152576446533, 1.658921480178833, 1.5425795316696167, 1.3835883140563965, 1.374611258506775, 1.283076524734497, 1.1873449087142944, 1.0305041074752808, 0.9198285937309265, 0.9714589715003967, 1.0685871839523315, 0.8317276239395142, 0.8206718564033508, 0.7756466865539551, 0.6918852925300598, 0.5552906394004822, 0.5671074390411377, 0.4668289124965668, 0.5393309593200684, 0.487480491399765, 0.39784517884254

In [8]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# plot the results
epochs_range = np.arange(len(history.history['accuracy']))
fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))

fig.add_trace(
    go.Scatter(
        x=epochs_range, y=history.history['accuracy'],
        name='Training', legendgroup='accuracy', mode='lines',  line=dict(color='blue'),
        legendgrouptitle_text='Accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs_range, y=history.history['val_accuracy'], mode='lines',  line=dict(color='green'),
    name='Validation', legendgroup='accuracy'), row=1, col=1)

fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['loss'], line=dict(color='blue'),
    mode='lines', showlegend=False), row=1, col=2)
fig.add_trace(
    go.Scatter(x=epochs_range, y=history.history['val_loss'], line=dict(color='green'),
    mode='lines', showlegend=False), row=1, col=2)


fig.update_layout(
    template='plotly_dark',
    title="Inception - CK+ Dataset",
    title_x=0.5
    ## xaxis_title="Epochs"
    ## yaxis_title="LR Value",)
)

# save and display the plot image
pio.write_image(fig, "./history.png", scale=6, width=1000, height=600)
fig.show()

In [9]:
# evaluate the model
loss, acc = model.evaluate(test_ds, verbose=2)

7/7 - 0s - loss: 0.2312 - accuracy: 0.9490 - 89ms/epoch - 13ms/step
